In [ ]:
from sklearn.model_selection import train_test_split # splitting the dataset
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, classification_report, auc, roc_auc_score # metrics
from joblib import dump
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt